# Graph RAG for Career Guidance

## Overview
This notebook implements a **Graph Retrieval-Augmented Generation (GraphRAG)** system for providing personalized career advice using a knowledge graph of job skills and relationships.

## What it does:
1. **Loads Knowledge Graph**: 67 nodes, 382 edges representing skills, job roles, and their relationships
2. **Loads Embeddings**:
   - Text embeddings (384-dim) for semantic similarity
   - Graph embeddings (128-dim) for structural patterns
3. **Creates Hybrid Retriever**: Combines text + graph embeddings (60% graph, 40% text weighting)
4. **Natural Language Query Processing**:
   - User asks career questions in plain English
   - LLM (Gemini) extracts relevant search terms
   - Retriever finds matching nodes, similar skills, and graph relationships
5. **Generates Personalized Roadmaps**: LLM creates career advice grounded in graph data

## Key Features:
- Uses **association rule mining** (lift, confidence scores) to find skill relationships
- **Multi-hop graph traversal** to discover skill combinations and prerequisites
- **Hybrid similarity search** combining semantic and structural patterns
- Provides **data-driven career roadmaps** with statistics from real job postings

## Example Query:
"I want to be an ML/AI engineer. I have experience in Python and Java. What should my roadmap look like?"

In [ ]:
from google import genai
import os
os.environ['GOOGLE_API_KEY'] = #your key here
# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()



AI identifies patterns in data to make decisions or predictions.


In [5]:
# ============================================================================
# Add this cell to your graphRag.ipynb notebook
# Make sure you already have: kg, text_embeddings, graph_embeddings, retriever
# ============================================================================

from graphRetriever import HybridGraphRetriever

def graphrag_query(question, retriever, model_name='gemini-2.5-flash', verbose=True):
    """Minimal GraphRAG: Query analysis → Your retriever → Answer generation"""

    if verbose:
        print(f"\n{'='*70}")
        print(f" {question}")
        print(f"{'='*70}")

    model = genai.GenerativeModel(model_name)

    # Step 1: Extract search terms
    prompt1 = f"""Extract  search terms from this career query for a job skills graph.
Question: "{question}. Extract terms specific to job type and skills"
Return comma-separated list only:"""

    response = model.generate_content(prompt1)
    terms = [t.strip() for t in response.text.strip().split(',')][:5]

    if verbose:
        print(f"\n Search: {', '.join(terms)}")

    # Step 2: Your retriever does the work (graph + embeddings + relationships)
    all_docs = []
    seen = set()

    for term in terms:
        docs = retriever.get_relevant_documents(term)
        for doc in docs:
            nid = doc.metadata.get('node_id', doc.page_content[:50])
            if nid not in seen:
                all_docs.append(doc)
                seen.add(nid)

    if not all_docs:
        return "No relevant information found."

    if verbose:
        print(f" Found {len(all_docs)} documents\n")

    # Step 3: Generate answer
    context = "\n\n".join([f"[{doc.metadata.get('source', 'INFO')}] {doc.page_content}"
                           for doc in all_docs[:15]])

    prompt2 = f"""Career advisor with skills knowledge graph.

CONTEXT:
{context}

QUESTION: {question}

Provide a road map to achieve the clients goal in teh given timeframe. Stay strictly grounded in the context and only provide recommendations based on the context provided. Give your roadmap so that the user can become most effective , order reccomendations by importance.

ANSWER:"""

    if verbose:
        print(" Generating...\n")

    response = model.generate_content(prompt2)

    if verbose:
        print("="*70)
        print(response.text)
        print("="*70)

    return response.text

# ============================================================================
# TEST IT:
# ============================================================================



import pickle
import google.generativeai as genai
from graphRetriever import HybridGraphRetriever

# Load knowledge graph
print("Loading knowledge graph...")
with open('knowledge_graph/career_skills_kg_latest.pkl', 'rb') as f:
    kg = pickle.load(f)
print(f"✓ Loaded: {len(kg.nodes())} nodes, {len(kg.edges())} edges")

# Load text embeddings
print("\nLoading text embeddings...")
with open('knowledge_graph/text_embeddings.pkl', 'rb') as f:
    text_data = pickle.load(f)
print(f"✓ Loaded: {text_data['embeddings'].shape} text embeddings")

# Load graph embeddings
print("\nLoading graph embeddings...")
with open('knowledge_graph/graph_embeddings.pkl', 'rb') as f:
    graph_embeddings = pickle.load(f)
print(f"✓ Loaded: {len(graph_embeddings)} graph embeddings")

# Create the retriever
retriever = HybridGraphRetriever(
    knowledge_graph=kg,
    node_ids=text_data['node_ids'],
    text_embeddings=text_data['embeddings'],
    graph_embeddings=graph_embeddings,
    alpha=0.6  # 60% graph, 40% text
)

result = graphrag_query(
    "I want to be an Backend engineer. I only have experience in SQL. What should my roadmap look like?",
    retriever
)

Loading knowledge graph...
✓ Loaded: 67 nodes, 382 edges

Loading text embeddings...
✓ Loaded: (175, 384) text embeddings

Loading graph embeddings...
✓ Loaded: 175 graph embeddings
  ✓ Created 175 weighted embeddings
✓ Retriever created with 175 nodes
  Alpha (α) = 0.6
  Text weight = 40%, Graph weight = 60%

 I want to be an Backend engineer. I only have experience in SQL. What should my roadmap look like?

 Search: Backend engineer, SQL

🔍 Query: 'Backend engineer'
   ✓ Found 1 matching node(s): ['Backend Engineer']
   ✓ Found 5 similar nodes
   ✓ Added 37 relationships
   ✓ Total documents: 9

🔍 Query: 'SQL'
   ✓ Found 1 matching node(s): ['SQL']
   ✓ Found 5 similar nodes
   ✓ Added 4 relationships
   ✓ Total documents: 7
 Found 11 documents

 Generating...

You currently have experience in SQL, which is a strong foundation as it is required by 36.7% of Backend Engineer positions analyzed. To become a more effective Backend Engineer, your roadmap should prioritize acquiring the fo